### Importing useful modules

In [1]:
import pandas as pd
import numpy as np

import calendar

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
# setting Jedha color palette as default
pio.templates["jedha"] = go.layout.Template(
    layout_colorway=["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
)
pio.templates.default = "jedha"
pio.renderers.default = "vscode"
from IPython.display import display

### File reading and basic exploration

In [2]:
# Importing dataset
df = pd.read_csv("Walmart_Store_sales.csv")

# Displaying the first rows of the dataset
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In order to predict weekly sales from other information in the dataset, we will consider the following variables:
- Target variable (Y): 'Weekly_Sales'.
- Explanatory variables (X): 'Store', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment'. The 'Date' column will be used to create columns 'Year', 'Month', 'Date', 'Day_of_the_Week' that will be used as explanatory variables (X).

*Extract from description of the original dataset on Kaggle:*

"This is the historical data that covers sales from 2010-02-05 to 2012-11-01, in the file Walmart_Store_sales. Within this file you will find the following fields:



- *Store* - the store number
- *Date* - the week of sales
- *Weekly_Sales* - sales for the given store
- *Holiday_Flag* - whether the week is a special holiday week 1 – Holiday week 0 – Non-holiday week
- *Temperature* - Temperature on the day of sale
- *Fuel_Price* - Cost of fuel in the region
- *CPI* - Prevailing consumer price index
- *Unemployment* - Prevailing unemployment rate


Holiday Events:
- Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13\
- Labour Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13\
- Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13\
- Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13"

In [3]:
#Checking data types in the dataframe
df.dtypes

Store           float64
Date             object
Weekly_Sales    float64
Holiday_Flag    float64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

We see that the 'Date' column contains values in string ('object') format. These dates will need to be converted to datetime format to be useful for analysis.

In [4]:
# Basic statistics
print("Number of rows : {}".format(df.shape[0])), 
print("Number of columns : {}".format(df.shape[1]))
print()

print("Basics statistics: ")
data_desc = df.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Number of rows : 150
Number of columns : 8

Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

Some ideas for dealing with missing values: 
- The rows with the missing values in the column 'Weekly_Sales' (target variable) will be dropped as they will not be useful for the model.
- If there is a row where the date is known but the holiday flag is missing, is may be possible to deduce the value of holiday flag based on the date (i.e. determine whether there was a holiday on a given week).
- It does not seem likely that missing dates could be easily inferred from the data in other columns.

### Visualizing the distribution of numeric features

In [5]:
num_features = ['Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
for i in range(len(num_features)):
    fig = px.histogram(df[num_features[i]])
    fig.show()

### Visualizing correlation between variables

In [6]:
# Converting store number to string data type so that it is not considered as a numeric variable
df['Store'] = df['Store'].apply(lambda x : str(x))

# Correlation matrix
corr_matrix = df.corr(numeric_only=True).round(2)

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.update_layout(margin = {'l': 120}) # to avoid cropping of column names


fig.show()

### Visualizing pairwise dependencies between variables

In [7]:
# Visualize pairwise dependencies
fig = px.scatter_matrix(df)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=800, width = 800)
fig.show()

C:\Users\yuliy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\plotly\express\_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



### Preprocessing with Pandas

In [8]:
# Dropping rows for which there is the value of the target variable 'Weekly_Sales' is not indicated
rows_to_keep = (~(df['Weekly_Sales'].isnull()))
df = df.loc[rows_to_keep,:].reset_index(drop = True)

In [9]:
# Converting values in the column 'Date' from object data type to datetime
df["Date"] = pd.to_datetime(df["Date"], format='%d-%m-%Y')

# Creating columns with information about year, month, day, week number and day of the week information
df["Year"] = pd.DatetimeIndex(df["Date"]).year
df["Month"] = pd.DatetimeIndex(df["Date"]).month
df["Day"] = pd.DatetimeIndex(df["Date"]).day
df["Week_Number"] = df["Date"].dt.isocalendar().week
df['Day_of_Week'] = pd.to_datetime(df['Date']).dt.day_name()

# Checking the resulting dataframe
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Week_Number,Day_of_Week
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,7,Friday
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,12,Friday
2,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,<NA>,NaN
3,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,21,Friday
4,4.0,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,21,Friday


It looks like the day of the week is always the same for all rows where the date is indicated. We will check this idea using pandas method 'nunique()'.

In [10]:
#checking the number of unique values for each column:
print(df.nunique())

Store            20
Date             79
Weekly_Sales    136
Holiday_Flag      2
Temperature     119
Fuel_Price      113
CPI             123
Unemployment    100
Year              3
Month            12
Day              30
Week_Number      45
Day_of_Week       1
dtype: int64


It does turn that all dates in the dataset fall on the same day of the week, Friday. I suppose that the reason for that that the 20 diffrerent stores included in the dataset are required to report weekly sales data on Fridays. This way, the date indicated for a row in the dataset is simply the date that corresponds to the Friday when a report was filed and in itself has no particular interest for explaining the amount of sales. The same goes for the day of the week as it is always fixed.

For this reason, it not useful to include the columns 'Date' and 'Day_of_Week' as the explanatory variables for training a machine learning model. 

'Date' column can still be used to infer some missing values for the column 'Holiday_Flag'. I will consider that the stores report their sales each Friday for the period including last week's Friday to this week's Thursday, but excluding this week's Friday (= the day of the report). So if a store files a weekly report on the 8th of February, and there was no holiday during the week covered by the report (i.e. from the 1st of February to the 7th of February included), the holiday flag for such week will be set to 0.

In [11]:
# Creating columns that will contain dates of the start and the end of each reporting period
df['Start_of_reporting_period'] = df['Date'] - pd.offsets.Week() # The beginnng of the reporting period is last week's Friday
df['End_of_reporting_period'] = df['Date'] - pd.offsets.Day() # The end of the reporting period is this week's Thursday

In [12]:
# Storing dates of holidays as a list of strings
holidays = ['12-Feb-10', '11-Feb-11', '10-Feb-12', '8-Feb-13', 
            '10-Sep-10', '9-Sep-11', '7-Sep-12', '6-Sep-13', 
            '26-Nov-10', '25-Nov-11', '23-Nov-12', '29-Nov-13', 
            '31-Dec-10', '30-Dec-11', '28-Dec-12', '27-Dec-13']

# Converting values in the column 'Date' from object data type to datetime
holidays = pd.to_datetime(holidays, format='%d-%b-%y')

# Checking the resulting list
print(holidays)
print()

# Checking whether the dates have been converted to timestamp type
print(type(holidays[0]))



DatetimeIndex(['2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08',
               '2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06',
               '2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29',
               '2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27'],
              dtype='datetime64[ns]', freq=None)

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [13]:
# Displaying the rows of the dataframe where the date is indicated but the holiday flag is not
display(df[~df['Date'].isna() & df['Holiday_Flag'].isna()])

# Storing indexes of the rows of interest in the variable
no_flag_indexes = df[~df['Date'].isna() & df['Holiday_Flag'].isna()].index

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Week_Number,Day_of_Week,Start_of_reporting_period,End_of_reporting_period
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,7,Friday,2011-02-11,2011-02-17
14,6.0,2010-04-30,1498080.16,NaN,68.91,2.780,211.894272,7.092,2010.0,4.0,30.0,17,Friday,2010-04-23,2010-04-29
40,7.0,2011-08-26,629994.47,NaN,57.60,3.485,194.379637,8.622,2011.0,8.0,26.0,34,Friday,2011-08-19,2011-08-25
45,1.0,2011-08-05,1624383.75,NaN,91.65,3.684,215.544618,7.962,2011.0,8.0,5.0,31,Friday,2011-07-29,2011-08-04
50,14.0,2011-03-25,1879451.23,NaN,41.76,3.625,184.994368,8.549,2011.0,3.0,25.0,12,Friday,2011-03-18,2011-03-24
67,1.0,2010-08-27,1449142.92,NaN,85.22,2.619,211.567306,7.787,2010.0,8.0,27.0,34,Friday,2010-08-20,2010-08-26
82,9.0,2010-07-09,485389.15,NaN,78.51,2.642,214.656430,6.442,2010.0,7.0,9.0,27,Friday,2010-07-02,2010-07-08
108,9.0,2010-06-18,513073.87,NaN,82.99,2.637,215.016648,6.384,2010.0,6.0,18.0,24,Friday,2010-06-11,2010-06-17
123,4.0,2011-07-08,2066541.86,NaN,84.59,3.469,129.112500,5.644,2011.0,7.0,8.0,27,Friday,2011-07-01,2011-07-07


In [14]:
# For rows of interest, we store dates of the start of the weekly reporting period 
# and of the end of the weekly reporting period as zip object.
weekly_periods = zip(df[~df['Date'].isna() & df['Holiday_Flag'].isna()]['Start_of_reporting_period'], 
                    df[~df['Date'].isna() & df['Holiday_Flag'].isna()]['End_of_reporting_period'])

In [15]:
holiday_flag_list =[any(x<=z and y>=z for z in holidays) for x, y in weekly_periods]

print(holiday_flag_list)

[True, False, False, False, False, False, False, False, False]


It looks like reporting period did indeed include a holiday. After checking the dataframe, the weekly sales report filed on the 18th of February 2011 supposedly covers the period from the 11th to the 17th of February 2011 included. The 11th February of 2011 was the day when Super Bowl was held.

In [16]:
# Using the holiday flag list to complete missing holiday flags:
for index, holiday_flag in zip(no_flag_indexes, holiday_flag_list):
    df.loc[index, 'Holiday_Flag'] = 1 if holiday_flag == True else 0


# Checking whether the previously emply values were filled correctly:
df.loc[df.index[no_flag_indexes]]

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Week_Number,Day_of_Week,Start_of_reporting_period,End_of_reporting_period
0,6.0,2011-02-18,1572117.54,1.0,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,7,Friday,2011-02-11,2011-02-17
14,6.0,2010-04-30,1498080.16,0.0,68.91,2.780,211.894272,7.092,2010.0,4.0,30.0,17,Friday,2010-04-23,2010-04-29
40,7.0,2011-08-26,629994.47,0.0,57.60,3.485,194.379637,8.622,2011.0,8.0,26.0,34,Friday,2011-08-19,2011-08-25
45,1.0,2011-08-05,1624383.75,0.0,91.65,3.684,215.544618,7.962,2011.0,8.0,5.0,31,Friday,2011-07-29,2011-08-04
50,14.0,2011-03-25,1879451.23,0.0,41.76,3.625,184.994368,8.549,2011.0,3.0,25.0,12,Friday,2011-03-18,2011-03-24
67,1.0,2010-08-27,1449142.92,0.0,85.22,2.619,211.567306,7.787,2010.0,8.0,27.0,34,Friday,2010-08-20,2010-08-26
82,9.0,2010-07-09,485389.15,0.0,78.51,2.642,214.656430,6.442,2010.0,7.0,9.0,27,Friday,2010-07-02,2010-07-08
108,9.0,2010-06-18,513073.87,0.0,82.99,2.637,215.016648,6.384,2010.0,6.0,18.0,24,Friday,2010-06-11,2010-06-17
123,4.0,2011-07-08,2066541.86,0.0,84.59,3.469,129.112500,5.644,2011.0,7.0,8.0,27,Friday,2011-07-01,2011-07-07


In [17]:
#walmart['Week_Number'] = walmart['Week_Number'].astype(float)

According to instructions for the project, it was equally necessary to drop rows containing in the columns 'Temperature', 'Fuel_price', 'CPI' and 'Unemployment'. For each of these columns, we will consider as outliers the values that are greater than or less than 3 standard deviations from the mean.

In [18]:
# Deleting rows with outliers
columns_with_outliers = ["Temperature", "Fuel_Price", "CPI", "Unemployment"]
for column in columns_with_outliers:
    mask_outliers = np.abs(df[column]-df[column].mean()) <= (3*df[column].std())
    mask_null = df[column].isnull()
    df = df[mask_outliers|mask_null]
    print(column)
    print(df.shape)

print()
print(df.shape)

Temperature
(136, 15)
Fuel_Price
(136, 15)
CPI
(136, 15)
Unemployment
(131, 15)

(131, 15)


In [19]:
print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Percentage of missing values: 


Store                         0.000000
Date                         13.740458
Weekly_Sales                  0.000000
Holiday_Flag                  1.526718
Temperature                  10.687023
Fuel_Price                    9.160305
CPI                           8.396947
Unemployment                 10.687023
Year                         13.740458
Month                        13.740458
Day                          13.740458
Week_Number                  13.740458
Day_of_Week                  13.740458
Start_of_reporting_period    13.740458
End_of_reporting_period      13.740458
dtype: float64

In [20]:
df['Store'].nunique()

19

In [21]:
# Checking how many missing values are still there in the dataframe:
print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Percentage of missing values: 


Store                         0.000000
Date                         13.740458
Weekly_Sales                  0.000000
Holiday_Flag                  1.526718
Temperature                  10.687023
Fuel_Price                    9.160305
CPI                           8.396947
Unemployment                 10.687023
Year                         13.740458
Month                        13.740458
Day                          13.740458
Week_Number                  13.740458
Day_of_Week                  13.740458
Start_of_reporting_period    13.740458
End_of_reporting_period      13.740458
dtype: float64

As seen before, we will drop the the 'Date' column as it does not contain information helpful for machine learning. We extracted some features from column with dates, but they are missing for the rows where the date was missing. Therefore, I chose to delete all rows where the date is missing.

In [22]:
df = df.dropna(subset=['Date'])

In [23]:
print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

df.shape

Percentage of missing values: 


Store                        0.000000
Date                         0.000000
Weekly_Sales                 0.000000
Holiday_Flag                 0.000000
Temperature                  8.849558
Fuel_Price                   9.734513
CPI                          7.964602
Unemployment                 9.734513
Year                         0.000000
Month                        0.000000
Day                          0.000000
Week_Number                  0.000000
Day_of_Week                  0.000000
Start_of_reporting_period    0.000000
End_of_reporting_period      0.000000
dtype: float64

(113, 15)

In [24]:
type(df)

pandas.core.frame.DataFrame

In [25]:
# Dropping columns that we will not use for machine learning
columns_to_drop = ['Date', 'Day', 'Day_of_Week', 'Start_of_reporting_period', 'End_of_reporting_period']
df = df.drop(columns_to_drop, axis=1)

df.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week_Number
0,6.0,1572117.54,1.0,59.61,3.045,214.777523,6.858,2011.0,2.0,7
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,12
3,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,21
4,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,21
5,15.0,695396.19,0.0,69.80,4.069,134.855161,7.658,2011.0,6.0,22


In [26]:
print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Percentage of missing values: 


Store           0.000000
Weekly_Sales    0.000000
Holiday_Flag    0.000000
Temperature     8.849558
Fuel_Price      9.734513
CPI             7.964602
Unemployment    9.734513
Year            0.000000
Month           0.000000
Week_Number     0.000000
dtype: float64

I chose to leave some missing values in the columns 'Temperature', 'Fuel_Price', 'CPI' and 'Unemployment'. We will try to impute them later.

As we have created some new features, let us check again if we can find some correlations between different explanatory variables as well as between explanatory variables and the target variable.

In [27]:
print(df['Month'].value_counts())
print(df['Week_Number'].value_counts())

6.0     14
2.0     12
5.0     12
7.0     12
3.0     10
8.0     10
4.0      9
10.0     8
11.0     8
12.0     7
9.0      6
1.0      5
Name: Month, dtype: int64
25    6
21    5
7     4
34    4
27    4
30    4
12    4
6     4
45    4
19    4
11    3
17    3
50    3
16    3
24    3
1     3
41    3
38    3
46    3
49    3
42    3
22    3
26    2
32    2
23    2
31    2
5     2
8     2
37    2
33    2
15    2
13    2
18    2
47    1
29    1
36    1
2     1
20    1
3     1
52    1
39    1
10    1
28    1
9     1
40    1
Name: Week_Number, dtype: Int64


In [28]:
# Correlation matrix
corr_matrix = df.corr(numeric_only=True).round(2)

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.update_layout(margin = {'l': 120}) # to avoid cropping of column names


fig.show()

It looks like there is perfect correllation between 'Month' and 'Week_Number'. It is not surprising as it is easy to find month knowing the number of the week, so one of the columns is redundant.

In [29]:
print("Number of rows for each month", df['Month'].value_counts())
print("Number of rows for each week number", df['Week_Number'].value_counts())

Number of rows for each month 6.0     14
2.0     12
5.0     12
7.0     12
3.0     10
8.0     10
4.0      9
10.0     8
11.0     8
12.0     7
9.0      6
1.0      5
Name: Month, dtype: int64
Number of rows for each week number 25    6
21    5
7     4
34    4
27    4
30    4
12    4
6     4
45    4
19    4
11    3
17    3
50    3
16    3
24    3
1     3
41    3
38    3
46    3
49    3
42    3
22    3
26    2
32    2
23    2
31    2
5     2
8     2
37    2
33    2
15    2
13    2
18    2
47    1
29    1
36    1
2     1
20    1
3     1
52    1
39    1
10    1
28    1
9     1
40    1
Name: Week_Number, dtype: Int64


I chose to drop the column 'Week_Number', as for some weeks numbers there is only one row, and overall there are fewer rows per week number is smaller than there are rows per month, so the 'Week_Number' column would be less useful for training.

There is also a strong corelation between 'Year' and 'Fuel_Price'. It means that these columns will give essentially the same information to the model, so one of the columns is not useful. I chose to drop the column 'Fuel_Price' as it has more missing values than 'Year'.

df['Month'] = pd.to_datetime(df['Month'], format = '%m').dt.month_name()

In [30]:
columns_to_drop = ['Week_Number', 'Fuel_Price']
df = df.drop(columns_to_drop, axis=1)

I have also chosen to treat month as a categorical variable. To make this more evident, the number of the month will be converted to the name of the month.

In [31]:
df['Month'] = pd.to_datetime(df['Month'], format = '%m').dt.month_name()

# Checking the resulting dataframe
df.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,CPI,Unemployment,Year,Month
0,6.0,1572117.54,1.0,59.61,214.777523,6.858,2011.0,February
1,13.0,1807545.43,0.0,42.38,128.616064,7.470,2011.0,March
3,6.0,1644470.66,0.0,78.89,212.412888,7.092,2010.0,May
4,4.0,1857533.70,0.0,NaN,126.160226,7.896,2010.0,May
5,15.0,695396.19,0.0,69.80,134.855161,7.658,2011.0,June


In [32]:
df['Month'].value_counts()

June         14
February     12
May          12
July         12
March        10
August       10
April         9
October       8
November      8
December      7
September     6
January       5
Name: Month, dtype: int64

In [33]:
# Separate target variable Y from features X
print("Separating labels from features...")
features_list = ['Store', 'Holiday_Flag', 'Temperature', 'CPI', 'Unemployment', 'Year', 'Month']
target_variable = "Weekly_Sales"

X = df.loc[:,features_list]
Y = df.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

Y : 
0    1572117.54
1    1807545.43
3    1644470.66
4    1857533.70
5     695396.19
Name: Weekly_Sales, dtype: float64

X :
  Store  Holiday_Flag  Temperature         CPI  Unemployment    Year     Month
0   6.0           1.0        59.61  214.777523         6.858  2011.0  February
1  13.0           0.0        42.38  128.616064         7.470  2011.0     March
3   6.0           0.0        78.89  212.412888         7.092  2010.0       May
4   4.0           0.0          NaN  126.160226         7.896  2010.0       May
5  15.0           0.0        69.80  134.855161         7.658  2011.0      June


### Preprocessing with Scikit-Learn

In [34]:
# Dividing dataset into Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [35]:
# Deciding which features (explanatory variables) will be treated as numeric and which features will be treated as categorical
numeric_features = ["Temperature", "CPI", "Unemployment", "Year"]
categorical_features = ["Store", "Holiday_Flag", "Month"]

print('Numeric features ', numeric_features)
print('Categorical features ', categorical_features)

Numeric features  ['Temperature', 'CPI', 'Unemployment', 'Year']
Categorical features  ['Store', 'Holiday_Flag', 'Month']


In [36]:
df['Store'].nunique()

19

In [37]:
df["Store"].value_counts()

3.0     10
13.0     9
14.0     9
1.0      8
19.0     8
5.0      7
18.0     7
7.0      7
2.0      6
6.0      6
8.0      6
4.0      6
17.0     5
20.0     4
16.0     4
9.0      4
15.0     3
10.0     3
11.0     1
Name: Store, dtype: int64

In order to impute missing values in the columns 'Temperature', 'Fuel_Price', 'CPI' and 'Unemployment'. I supposed that the different stores are included in the dataset are situated in different places. So in order to find the temperature for a given shop, it is best to take temperature from a similar shop than to calculate average temperature for all shops.

(If the geographical position of the stores were know, it might be possible to infer temperature on a certain date from historic data or from h )

In [38]:
# Create pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer()),
    ('scaler', StandardScaler())
])

In [39]:
# Create pipeline for categorical features
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [40]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [41]:
type(X_train)

pandas.core.frame.DataFrame

In [42]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5]) # MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head()) 
X_test = preprocessor.transform(X_test) 
# we made based on the training set, therefore we can only apply transformations that were parametered using the training set.
# Otherwise this creates what is called a leak from the test set which will introduce a bias in all your results.
print('...Done.')
print(X_test[0:5,:]) # MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
print()

Performing preprocessings on train set...
    Store  Holiday_Flag  Temperature         CPI  Unemployment    Year  \
49    9.0           0.0        85.02  214.896576           NaN  2010.0   
10   18.0           0.0        52.02  132.763355         9.331  2010.0   
97   19.0           0.0        72.83  132.598387         8.099  2010.0   
101  20.0           1.0        28.85  204.643227         7.484  2010.0   
69   20.0           0.0        75.17  204.567546         7.856  2010.0   

        Month  
49       June  
10    October  
97       July  
101  December  
69       June  
...Done.
  (0, 0)	1.5380947856614469
  (0, 1)	0.9862332475068893
  (0, 2)	-0.19809831459266816
  (0, 3)	-1.055587152274032
  (0, 21)	1.0
  (0, 28)	1.0
  (1, 0)	-0.37699873299357567
  (1, 1)	-1.1402273433969634
  (1, 2)	1.9960067235770353
  (1, 3)	-1.055587152274032
  (1, 11)	1.0
  (1, 32)	1.0
  (2, 0)	0.8306708464976673
  (2, 1)	-1.1444984204277335
  (2, 2)	0.6931643257434726
  (2, 3)	-1.055587152274032
  (2, 12)	

In [43]:
type(X_train)

scipy.sparse._csr.csr_matrix

In [44]:
type(X_test)

scipy.sparse._csr.csr_matrix

### Train model

In [45]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

Train model...
...Done.


### Performance assessment

In [46]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = regressor.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()

Predictions on training set...
...Done.
[ 515471.73558197 1034951.66774337 1438338.97908417 2099305.79506901
 1880431.60954587 1887381.7205781  1885844.17556147 1563061.40411324
  499806.30146959 2562661.43359814 1519213.75021917 1980091.55422632
 1896775.71142736 1964598.06889089 1312011.07934893 1766306.51583799
  520475.41484102 1370663.44060689 1357797.91252765  845887.82548236
 2109245.28229366  374810.32071217 1873567.2617098   431164.83064087
 1551646.0655003  1852989.67565737 2001908.12173651 1868971.49785548
 2056744.05318915  670198.24940465  469098.56548791 1084695.12913284
  301055.97661578  511815.93979389 1418381.98103123 1996658.54429627
 2620449.10660612  416318.54774025 1576788.37000702 1609574.68318909
  571818.74727659  307883.745421    503773.14001643 1443839.67942263
  759993.28000654  331302.15136281 1899910.186612   2396095.32240462
 1757242.97646872  413307.71137629 1018356.33805541 1525538.25806782
 2387347.53897044  267912.16660914 1368439.57850423  972813.696

In [47]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = regressor.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()

Predictions on test set...
...Done.
[ 319798.27075431 1515760.15278948 1559409.99296059  938315.60547625
  409954.47446404 1533967.03146006 1888227.28867386 2493550.51534919
 1970977.34021122 1632745.87612055  948896.6756819  1951401.5793346
 1075403.23499561  419594.10058992  609874.53088892  115622.2364594
  491543.17252015  271268.35856532 1853107.45503529  499480.5407953
 1995933.47284372  580988.26760635 2163686.78569469]



In [48]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9856778239903055
R2 score on test set :  0.971924539242769


### Interpreting the model's coefficients
As we've standardized our features, we can use the coefficients of the regression to estimate the importance of each feature for the prediction. The model's parameters are saved in a `.coef_` attribute:

In [49]:
regressor.coef_

array([   52511.91860632,   -48002.76328651,   -77900.79283338,
         -11770.60209014,   283177.37621644,   314362.90691481,
         285818.95311021,   533499.94475064,  -997419.98152382,
       -1081093.92867461,  -826853.55573399,  -403641.82241198,
        -137278.32300406,   301776.5270968 ,   284876.59912776,
       -1219397.91469825,   311117.58144926, -1369366.32620682,
          17540.04082654,  -826282.54633855,  -687250.36461513,
       -1146675.12080727,  -200548.7577398 ,    36023.71122465,
         644998.39464436,   304412.43263671,    52154.2746363 ,
          32737.51553024,   114969.8230957 ,    74841.38594881,
          70095.96916803,   167890.44260301,    60828.22948679,
          40971.41460894])

In [50]:
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Temperature', 'CPI', 'Unemployment', 'Year', 'Store_10.0', 'Store_11.0', 'Store_13.0', 'Store_14.0', 'Store_15.0', 'Store_16.0', 'Store_17.0', 'Store_18.0', 'Store_19.0', 'Store_2.0', 'Store_20.0', 'Store_3.0', 'Store_4.0', 'Store_5.0', 'Store_6.0', 'Store_7.0', 'Store_8.0', 'Store_9.0', 'Holiday_Flag_1.0', 'Month_August', 'Month_December', 'Month_February', 'Month_January', 'Month_July', 'Month_June', 'Month_March', 'Month_May', 'Month_November', 'Month_October', 'Month_September']


In [51]:
# Create a pandas DataFrame
coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])
coefs.head(10)

,coefficients
Temperature,5.251192e+04
CPI,-4.800276e+04
Unemployment,-7.790079e+04
Year,-1.177060e+04
Store_10.0,2.831774e+05
Store_11.0,3.143629e+05
Store_13.0,2.858190e+05
Store_14.0,5.334999e+05
Store_15.0,-9.974200e+05
Store_16.0,-1.081094e+06


In [52]:
# Compute abs() and sort values
feature_importance = abs(coefs).sort_values(by = 'coefficients', ascending=False)
print("10 most important features:")
display(feature_importance.head(10))

10 most important features:


,coefficients
Store_5.0,1.369366e+06
Store_3.0,1.219398e+06
Store_9.0,1.146675e+06
Store_16.0,1.081094e+06
Store_15.0,9.974200e+05
Store_17.0,8.268536e+05
Store_7.0,8.262825e+05
Store_8.0,6.872504e+05
Month_December,6.449984e+05
Store_14.0,5.334999e+05


In [53]:
# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.show()

In [54]:
# Performing 10-fold cross-validation to evaluate the generalized R2 score obtained with a Ridge model
print("10-fold cross-validation...")
regressor_ridge = Ridge()
scores_ridge = cross_val_score(regressor_ridge, X_train, Y_train, cv=10)
print('The cross-validated R2-score is : ', scores_ridge.mean())
print('The standard deviation is : ', scores_ridge.std())

10-fold cross-validation...
The cross-validated R2-score is :  0.8436908200643117
The standard deviation is :  0.05902573975380519


In [55]:
# Performing grid search
print("Grid search...")

# Grid of values to be tested
params_ridge = {
    'alpha': [0.0, 0.1, 0.5, 1.0]
    #'alpha': [0.05, 0.07, 0.1, 0.13, 0.15]
}
gridsearch_ridge = GridSearchCV(regressor_ridge, param_grid = params_ridge, cv = 10) # cv : the number of folds to be used for CV
gridsearch_ridge.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_ridge.best_params_)
print("Best R2 score : ", gridsearch_ridge.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.1}
Best R2 score :  0.915921084407729


In [56]:
# Perform 10-fold cross-validation to evaluate the generalized R2 score obtained with a Ridge model
print("10-fold cross-validation...")
regressor_lasso = Lasso()
scores_lasso = cross_val_score(regressor_lasso, X_train, Y_train, cv=10)
print('The cross-validated R2-score is : ', scores_lasso.mean())
print('The standard deviation is : ', scores_lasso.std())

10-fold cross-validation...
The cross-validated R2-score is :  0.9158348340380309
The standard deviation is :  0.11025991175158374


In [57]:
# Perform grid search
print("Grid search...")

# Grid of values to be tested
params_lasso = {'alpha' : [500, 750, 1000, 1250, 5000]
                #'alpha' : [0.1, 1, 10, 100, 1000, 10000]
                }
gridsearch_lasso = GridSearchCV(regressor_lasso, param_grid = params_lasso, cv = 10) # cv : the number of folds to be used for CV
gridsearch_lasso.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_lasso.best_params_)
print("Best R2 score : ", gridsearch_lasso.best_score_)

# max_iter: pas assez de données pour converger vers la bonne solution; il y a une limite par défaut 
# qui évite de faire une boucle infinie
# essayer max_iter = 10000

Grid search...
...Done.
Best hyperparameters :  {'alpha': 1250}
Best R2 score :  0.9246908744741107


In [58]:
# Perform grid search
print("Grid search...")

# Grid of values to be tested
params_lasso = {
    #'alpha' : [200, 300, 700, 800, 900, 1000, 5000]
    #'alpha': [10, 15, 20, 100]
    #'alpha': [0.1, 0.5, 1.0]
    #'alpha': [0.5, 1.0, 1.5, 2]
}
gridsearch_lasso = GridSearchCV(regressor_lasso, param_grid = params_lasso, cv = 10) # cv : the number of folds to be used for CV
gridsearch_lasso.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_lasso.best_params_)
print("Best R2 score : ", gridsearch_lasso.best_score_)

# max_iter: pas assez de données pour converger vers la bonne solution; il y a une limite par défaut 
# qui évite de faire une boucle infinie
# essayer max_iter = 10000

Grid search...
...Done.
Best hyperparameters :  {}
Best R2 score :  0.9158348340380309
